In [36]:
import os

from dotenv import load_dotenv
from langchain.llms import OpenAI

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [ ]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.3)
name = llm(
    "I want to open a restaurant for Indian food. Suggest a fancy name for this!")
print(name)

In [35]:
from langchain import HuggingFaceHub
llm_huggingface = HuggingFaceHub(
    repo_id="google/flan-t5-large", model_kwargs={"temperature": 0, "max_length": 64})

In [7]:
output = llm_huggingface.predict("Can you tell me the capital of Russia?")
print(output)

moscow


In [6]:
output = llm_huggingface.predict("Can you write a poem about AI")
print(output)

i love the way i look at the world i love the way i feel i love the way i think i feel i love the way i feel i love the way i think i feel i love the way i feel i love the way 


In [10]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(input_variables=[
                                      'country'], template="Tell me the capital of {country}")

prompt_template_name.format(country='India')

'Tell me the capital of India'

In [13]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm_huggingface, prompt=prompt_template_name)
chain.run('South Africa')

'cape town'

In [15]:
capital_template = PromptTemplate(input_variables=['country'],
                                  template="Please tell me the capital of {country}")

capital_chain = LLMChain(llm=llm_huggingface, prompt=capital_template)

famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm_huggingface, prompt=famous_template)

In [20]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.run("India")

'The Temple of Lord Vishnu'

In [23]:
capital_template = PromptTemplate(input_variables=['country'],
                                  template="Please tell me the capital of {country}")

capital_chain = LLMChain(
    llm=llm_huggingface, prompt=capital_template, output_key="capital")

In [24]:
famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm_huggingface,
                        prompt=famous_template, output_key="places")

In [25]:
from langchain.chains import SequentialChain

chain = SequentialChain(chains=[capital_chain, famous_chain],
                        input_variables=['country'],
                        output_variables=['capital', "places"])

In [29]:
chain({'country': "India"})

{'country': 'India',
 'capital': 'chennai',
 'places': 'The Temple of Lord Vishnu'}

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [31]:
chatllm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.6, model='gpt-3.5-turbo')

c:\Users\mbantwal\Git Repositories\Personal\LangChain Tutorials\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
chatllm([
    SystemMessage(content="Yor are a comedian AI assitant"),
    HumanMessage(content="Please provide some comedy punchlines on AI")
])

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [37]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(",")

In [38]:
template = "Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [39]:
chain = chatprompt | chatllm | Commaseperatedoutput()

In [ ]:
chain.invoke({"text":"intelligent"})